In [1]:
import boto3
import re
import os
import time
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import sagemaker as sage
from time import gmtime, strftime


In [3]:
role = get_execution_role()
sess = sage.Session()

In [4]:
# generating the image path
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-word2vec:latest'.format(account, region)


In [6]:
# to deploy the model, you need to have the model created based on your model artifacts, create an endpoint configuration and then create the endpoint based on the two
# first let's create the model

client = boto3.client('sagemaker')
ModelName='TestCx-BYOA' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
first_model = client.create_model(
    ModelName=ModelName,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': 's3://sagemaker-us-east-1-639634733305/output/sagemaker-word2vec-2020-01-10-20-32-37-771/output/model.tar.gz'    # note that the model.tar.gz file is a tarball of your word2vec_2.model file
        },
    ExecutionRoleArn=role)

In [8]:
# create the endpoint configuration
endpoint_config_name = 'TestCx-BYOA-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':ModelName,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


TestCx-BYOA-2020-01-11-04-52-52
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:639634733305:endpoint-config/testcx-byoa-2020-01-11-04-52-52


In [9]:
# create the endpoint


endpoint_name = 'TestCx-BYOA-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

TestCx-BYOA-2020-01-11-04-52-59
arn:aws:sagemaker:us-east-1:639634733305:endpoint/testcx-byoa-2020-01-11-04-52-59
